# Incorporating RTS Data (AWS Console)

> *This notebook should work well in the `Python 3 (Data Science)` kernel in SageMaker Studio, or `conda_python3` in SageMaker Notebook Instances*

**In this notebook** we'll use the [**Amazon Forecast Console**](https://console.aws.amazon.com/forecast/home) to:

- Import the prepared *Related Time-Series* data from notebook 3 to our existing Amazon Forecast *Dataset Group*
- Train new predictors and generate forecasts using the additional data
- Explore how the extra information affects our forecast quality

Check out Notebook [4b. Incorporating RTS Data (SDK)](4b.%20Incorporating%20RTS%20Data%20(Python%20SDK).ipynb) for an alternative guide through the same steps from code using the **AWS Python SDK** instead!

To get started, navigate to the [Amazon Forecast Console](https://console.aws.amazon.com/forecast/home) and select the same *Dataset Group* you created in the earlier notebooks.

## Defining the RTS Dataset

Since our Dataset Group is already created, the first step will be to add Related Time-Series data will be to define the structure of the dataset.

▶️ **Open** the *Datasets* tab in the sidebar, **select** the *Related Time-Series* slot and **click** *Upload dataset* to start the process

![Datasets list screenshot with Related Time-Series slot selected](static/imgs/guide/rts-00-datasets-list.png)

In the *Create related time-series dataset* window that opens, set:

- The **Dataset name** as `forecast_poc_rts`
- The **Frequency** of the data as `hourly`, as we did before
- A data **schema** as below (check this matches the data as shown at the end of the RTS preparation notebook!):

```json
{
    "Attributes": [
        {
            "AttributeName": "timestamp",
            "AttributeType": "timestamp"
        },
        {
            "AttributeName": "temperature",
            "AttributeType": "float"
        },
        {
            "AttributeName": "rain_1h",
            "AttributeType": "float"
        },
        {
            "AttributeName": "snow_1h",
            "AttributeType": "float"
        },
        {
            "AttributeName": "clouds_all",
            "AttributeType": "float"
        },
        {
            "AttributeName": "weather",
            "AttributeType": "string"
        },
        {
            "AttributeName": "item_id",
            "AttributeType": "string"
        }
    ]
}
```

Your screen should look something like the below:

![Create RTS schema screenshot](static/imgs/guide/rts-01-rts-schema.png)

▶️ **Click** *Next* when you've checked these entries and are ready to move on

## Importing the RTS Data

In the next screen, you'll be asked to create an RTS import job from S3, just like we did for the TTS dataset.

Since this requires the Amazon Forecast service to access the Amazon S3 bucket, this is where we need the **service role** created in Notebook 0: Which has access to the target bucket and trusts the Forecast service.

You can run the below cell to read your stored role ARN, or if you don't have such a role set up in your account yet, you'll be able to **create a new role** with permissions to access the required bucket, through the UI:

In [ ]:
%store -r forecast_role_arn
print(forecast_role_arn)

we'll also need the **S3 URI** of the *RTS* file we uploaded in the previous notebook, which was printed out near the end but you can view again by running the below:

In [ ]:
%store -r related_s3uri
print(related_s3uri)

Remember that every **dataset import job** we create in Forecast is a batch **overwriting** process that clears out any pre-existing data in the dataset: *not* appending data to existing records.

To configure your initial import, enter:

- A **Name** for the job - we'll use `poc_import_rts`
- The **Timestamp Format** in the data, since some variations are permitted according to the [dataset guidelines](https://docs.aws.amazon.com/forecast/latest/dg/dataset-import-guidelines-troubleshooting.html) - our data uses the default
- The **IAM Role** we created earlier
- The **S3 URI** of the data we uploaded

Once you've entered and checked the values, click *Start import*

![Import RTS Data Screenshot](static/imgs/guide/rts-02-create-rts-import.png)

When the import is started, you'll likely be forwarded to the RTS dataset detail page, as below. The dataset may initially show as "Active" status before reverting to "Update in progress..." once the import starts.

![RTS Dataset Detail screen with import in progress](static/imgs/guide/rts-03-rts-importing.png)

▶️ We'll need to **wait** for this import to complete to move on to the next step

> ⏰ The import process can **take a little time** (on the order of ~10-15 minutes for our sample dataset) because of validation, filling & aggregation, and the overhead of spinning up infrastructure to execute the import

On small datasets like this, overheads can dominate the run-time and you should expect much better-than-linear scaling as dataset size is increased from this level.

## Creating and Training Predictors

Once the import is complete, your *Datasets* list page should show both the TTS and RTS datasets with *Active* status, as below:

![Datasets list screenshot showing RTS and TTS Active](static/imgs/guide/rts-04-datasets-rts-ready.png)

We can now create **Predictors** just as we did before, and our new models will automatically consume the updated datasets with RTS added in.

As detailed in the [Related Time-Series documentation](https://docs.aws.amazon.com/forecast/latest/dg/related-time-series-datasets.html#related-time-series-historical-futurelooking), certain algorithms including ARIMA are **not able** to make use of RTS data: So we'll create new versions of our *Prophet* and *DeepAR+* models to see how performance is affected, but won't re-train the ARIMA model as the results would be the same.

▶️ Navigate to the **Predictors** tab via the sidebar menu and click **Train new predictor**

▶️ In the "*Train Predictor*" screen, **enter**:

- The **Predictor name** as `forecast_poc_deeparp_rts`
- The **Forecast horizon** and **Forecast frequency** as `240` and `hour` (a 10-day, hourly forecast).
    - These settings are the same as for the initial, TTS-only models
- **Manual algorithm** selection and **Prophet** for this first model
- Set the **Country for holidays** to `United States`, since this data is from America and it's a country the Forecast service supports
- Leave any other settings as default

![Create Predictor screenshot for Prophet config, part 1](static/imgs/guide/rts-05.1-create-prophet-pt1.png)

![Create Predictor screenshot for Prophet config, part 2](static/imgs/guide/rts-05.2-create-prophet-pt2.png)

▶️ Once you've checked everything, click **Train predictor** to kick off the model training.

▶️ From the *Predictors* tab, **repeat the process** to create a predictor with:

- Name `forecast_poc_prophet_rts`
- The **DeepAR+** algorithm selected
- All other settings as per the Prophet predictor

You should now have 2 new predictors training, as shown in the predictors list screenshot below:

![Predictors list screen with 3 active, 2 training predictors](static/imgs/guide/rts-06-predictors-training.png)

▶️ Again you'll need to **wait** for at least the first predictor to finish training (enter `Active` status) before continuing

> ⏰ Predictor training can **take some time**: Simpler algorithms like ARIMA or ETS will typically train faster (may be ready in ~20mins on this example dataset), whereas more complex algorithms like DeepAR+ will usually take longer (may be approx 1hr on this example dataset)

## Examining the Predictors

Once the RTS-enriched predictors are `Active`, we can can compare them to the previous round of TTS-only predictors using the same metrics as discussed before. Summary metrics are shown in the predictors list view, as shown below (your exact results may vary):

![Predictors list screen showing 5 trained models](static/imgs/guide/rts-07-predictors-trained.png)

In our test run, both Prophet and DeepAR+ models generally improved with the addition of the weather data:

- Prophet's MAPE/wQL0.5 improved from 0.4847 to 0.4414; RMSE from ~1900 to ~1760; wQL0.9 from 0.2682 to 0.2456
- DeepAR+'s MAPE/wQL0.5 improved from 0.3199 to 0.3192; wQL0.9 from 0.2217 to 0.1974; wQL0.1 from 0.2045 to 0.1868

...However, the improvements were generally pretty marginal and some aspects were degraded (DeepAR+'s RMSE and Prophet's wQL0.1 both increased slightly).

It's likely that in this dataset there are other hidden factors with significant effects on the observed traffic flow - and possibly also that the weather features could be further consolidated/simplified to deliver clearer signals for forecasting.

## All Done!

In this notebook, we updated our dataset group with a *Related Time-Series* dataset of additional (weather) data to try and improve the forecast from the initial baseline using the traffic volume history alone.

You can refer to the previous notebooks 2a and 2b for guidance on visualizing the forecasts in the console and exporting + downloading them to compare against actual validation data.

Identifying important related data such as stock availability, holiday & promotion calendars, pricing and similar can have dramatic impacts on real-world forecasting use cases; but it's just as important to understand any gaps, errors, or aggregations in your datasets to check that your models are interpreting your data as you expect.

Check out the [Cleanup notebook](Cleanup.ipynb) for guidance on cleaning up your Amazon Forecast environment and also your Amazon S3 and AWS IAM setup from these experiments.

If you've prepared your own data for import to Amazon Forecast, you might also be interested in the [Data Diagnostic notebook](Data%20Diagnostic.ipynb) which can help run some basic checks and graphs on your data.